In [7]:
# --- indicators_metrics.ipynb ---

import pandas as pd
import numpy as np

# 1. Load data
from pathlib import Path
DATA_DIR = Path("data")
ohlcv = pd.read_csv(DATA_DIR / "ohlcv.csv", parse_dates=["date"])
ohlcv = ohlcv.sort_values(["ticker", "date"])

# 2. Compute indicators for all tickers
indicators = []

universe = ohlcv["ticker"].unique()
for ticker in universe:
    g = ohlcv[ohlcv["ticker"] == ticker].copy()
    g = g.sort_values("date")
    g["sma20"] = g["close"].rolling(20).mean()
    g["sma50"] = g["close"].rolling(50).mean()
    g["ema20"] = g["close"].ewm(span=20, adjust=False).mean()
    g["ema50"] = g["close"].ewm(span=50, adjust=False).mean()
    # RSI14
    delta = g["close"].diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(14).mean()
    avg_loss = loss.rolling(14).mean()
    rs = avg_gain / (avg_loss + 1e-10)
    g["rsi14"] = 100 - (100 / (1 + rs))
    # MACD (12/26/9)
    ema12 = g["close"].ewm(span=12, adjust=False).mean()
    ema26 = g["close"].ewm(span=26, adjust=False).mean()
    g["macd_line"] = ema12 - ema26
    g["macd_signal"] = g["macd_line"].ewm(span=9, adjust=False).mean()
    g["macd_hist"] = g["macd_line"] - g["macd_signal"]
    # Rolling vol
    g["vol20"] = g["close"].pct_change().rolling(20).std() * np.sqrt(252)
    g["vol60"] = g["close"].pct_change().rolling(60).std() * np.sqrt(252)
    # 60-day rolling correlation to SPY
    if "SPY" in universe and ticker != "SPY":
        df_merge = g[["date", "close"]].merge(
            ohlcv[ohlcv["ticker"] == "SPY"][["date", "close"]].rename(columns={"close": "close_spy"}),
            on="date", how="left"
        )
        g["roll_corr60"] = (
            g["close"].rolling(60).corr(df_merge["close_spy"])
        )
    else:
        g["roll_corr60"] = np.nan
    # Keep only needed columns
    cols = [
        "ticker", "date", "sma20", "sma50", "ema20", "ema50", "rsi14",
        "macd_line", "macd_signal", "macd_hist",
        "vol20", "vol60", "roll_corr60"
    ]
    indicators.append(g[cols])

# 3. Concatenate and save
indicators_df = pd.concat(indicators).sort_values(["ticker", "date"]).reset_index(drop=True)
outpath = DATA_DIR / "indicators.csv"
indicators_df.to_csv(outpath, index=False)
print(f"Indicators saved to: {outpath}")

# 4. Preview
indicators_df[indicators_df['ticker']=='MSFT'].sample(10)

Indicators saved to: data\indicators.csv


,ticker,date,sma20,sma50,ema20,ema50,rsi14,macd_line,macd_signal,macd_hist,vol20,vol60,roll_corr60
61751,MSFT,1996-08-29 04:00:00+00:00,7.758203,7.559121,7.711085,7.573166,48.648675,0.069081,0.076626,-0.007544,0.174779,0.277038,NaN
63588,MSFT,2003-12-17 05:00:00+00:00,26.025500,26.770800,26.350447,26.579000,82.716048,0.144225,-0.055569,0.199794,0.149912,0.262463,NaN
66984,MSFT,2017-06-15 04:00:00+00:00,70.152499,68.481600,70.188317,68.736655,51.666658,0.614262,0.828616,-0.214354,0.155856,0.133146,NaN
59936,MSFT,1989-06-27 04:00:00+00:00,0.398004,0.389948,0.391176,0.388859,16.902765,-0.003815,-0.000018,-0.003797,0.293502,0.320961,NaN
64557,MSFT,2007-10-24 04:00:00+00:00,30.155000,29.236600,30.217249,29.717361,67.420832,0.461906,0.390666,0.071241,0.184337,0.193303,NaN
59249,MSFT,1986-10-08 04:00:00+00:00,0.101606,0.101432,0.102427,0.102370,66.665243,0.000895,-0.000144,0.001039,0.408608,0.383670,NaN
62811,MSFT,2000-11-09 05:00:00+00:00,31.920313,31.577500,32.924709,32.418637,64.086687,1.374602,0.968512,0.406090,0.829991,0.540196,NaN
59643,MSFT,1988-04-29 04:00:00+00:00,0.385938,0.403941,0.386876,0.392739,37.999393,-0.005478,-0.006242,0.000764,0.369802,0.378941,NaN
68190,MSFT,2022-03-30 04:00:00+00:00,294.939500,297.428000,300.264952,301.028299,72.728714,4.202721,1.124379,3.078342,0.341462,0.332510,NaN
61409,MSFT,1995-04-25 04:00:00+00:00,4.577149,4.310117,4.616670,4.383550,78.321645,0.147738,0.119040,0.028699,0.334655,0.267256,NaN
